In [10]:
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
import os
import glob

In [11]:
def preprocess_image(image):
    resized_image = cv2.resize(image, (128, 128), interpolation=cv2.INTER_AREA)
    normalized_image = resized_image / 255.0
    return normalized_image

In [12]:
def build_cnn(input_shape):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(len(labels), activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [13]:
dataset_path = 'Dataset'
labels = ['50', '100', '200', '500']
data = []
labels_data = []

for label in labels:
    images_path = glob.glob(os.path.join(dataset_path, label, '*.jpg'))
    for image_path in images_path:
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        image = preprocess_image(image)
        data.append(image)
        labels_data.append(label)

In [8]:
data = np.array(data).reshape(-1, 128, 128, 1)
labels_data = np.array([labels.index(label) for label in labels_data])
labels_data = to_categorical(labels_data)

X_train, X_test, y_train, y_test = train_test_split(data, labels_data, test_size=0.25)

input_shape = X_train.shape[1:]
cnn_model = build_cnn(input_shape)
cnn_model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

Epoch 1/10
9/9 [==============================] - 3s 239ms/step - loss: 1.4295 - accuracy: 0.2982 - val_loss: 1.3806 - val_accuracy: 0.3229
Epoch 2/10
9/9 [==============================] - 2s 193ms/step - loss: 1.3674 - accuracy: 0.2982 - val_loss: 1.3709 - val_accuracy: 0.4271
Epoch 3/10
9/9 [==============================] - 2s 177ms/step - loss: 1.2919 - accuracy: 0.4561 - val_loss: 1.3127 - val_accuracy: 0.4167
Epoch 4/10
9/9 [==============================] - 2s 194ms/step - loss: 1.1916 - accuracy: 0.5193 - val_loss: 1.2103 - val_accuracy: 0.4688
Epoch 5/10
9/9 [==============================] - 2s 200ms/step - loss: 1.0721 - accuracy: 0.5825 - val_loss: 1.1738 - val_accuracy: 0.4271
Epoch 6/10
9/9 [==============================] - 2s 221ms/step - loss: 0.8983 - accuracy: 0.6281 - val_loss: 1.1450 - val_accuracy: 0.4375
Epoch 7/10
9/9 [==============================] - 2s 190ms/step - loss: 0.7821 - accuracy: 0.6877 - val_loss: 0.9250 - val_accuracy: 0.5625
Epoch 8/10
9/9 [====

In [9]:
cnn_model.save('model.h5')